# LoRA

Revisión informal de "_LoRA: Low-Rank Adaptation of Large Language Models_".

Técnica publicada en junio de 2021.

- [https://arxiv.org/pdf/2106.09685](https://arxiv.org/pdf/2106.09685)

Permite entrenar un modelo con datos nuevos sin tener que reentrenar el modelo completo.

## 1. Arquitectura

Los modelos realizan predicciones utilizando matrices de pesos aprendidas durante el entrenamiento con grandes conjuntos de datos.

```
Weights x Input ─> Output
```

_LoRA_ añade matrices de pesos entrenadas con nuevos conjuntos de datos que contribuyen a la predicción del modelo.

```
Weights x Input + (Weights' x Input) ─> Output
```

Los pesos originales del modelo se congelan durante el reentrenamiento y sólo se aprenden los nuevos pesos, con dimensiones significativamente menores.

## 2. Análisis

Analiza el proceso habitual utilizado para reentrenar un modelo, expone el concepto de rango de una matriz, y propone utilizar matrices de bajo rango para añadir conocimiento a un modelo preentrenado.

### 2.1. Función Objetivo

El marco teórico del _paper_ parte de un modelo de lenguaje autorregresivo preentrenado. Es decir, un sistema que modela la distribución de probabilidad condicional de una secuencia de salida $y$, dada una secuencia de entrada $x$, en función de unos parámetros $\Phi$ (pesos) aprendidos durante el preentrenamiento con un gran conjunto de datos.

- $ P_{\Phi}(y \mid x) $

Modelo que puede entrenarse posteriormente (_fine-tuning_) con nuevos conjuntos de datos (_datasets_). Colecciones de pares de secuencias de _tokens_ de entrada $x_i$ (_context_), con su correspondiente secuencia de salida $y_i$ (_target_).

- $ \mathcal{Z} = \{(x_i, y_i)\}_{i=1,\ldots,N} $

Las técnicas tradicionales reentrenan los modelos cargando los pesos preentrenados $\Phi_0$ y los actualizan a $\Phi_0 + \Delta \Phi$. Estableciendo el objetivo habitual de encontrar los nuevos parámetros $\Phi$ que maximicen la log-verosimilitud total, expresada como la suma de las log-probabilidades condicionales de cada _token_ de salida $y_t$, dada una secuencia de entrada $x$ y los _tokens_ anteriores generados $y_{<t}$.

- $ \underset{\Phi}{\text{max}} \sum\limits_{x,y \in \mathcal{Z}} \sum\limits_{t=1}^{\mid y \mid} \log \left(P_{\Phi}(y_t \mid x, y_{< t})\right) $

Esa aproximación resulta poco práctica para modelos de gran tamaño, ya que genera un nuevo conjunto de pesos tan grande como el original.

_LoRA_ propone actualizar los pesos con un incremento $\Delta \Phi(\Theta)$ calculado a partir de un nuevo conjunto de parámetros $\Theta$ mucho más reducido que el original $\mid \Theta \mid \ll \mid \Phi_0 \mid$. De forma que el objetivo sea encontrar los parámetros $\Theta$ que, al aplicar este incremento $\Delta \Phi(\Theta)$ al modelo original, maximicen la misma función de log-verosimilitud condicional.

- $ \underset{\Theta}{\text{max}} \sum\limits_{x,y \in \mathcal{Z}} \sum\limits_{t=1}^{\mid y \mid} \log \left(p_{\Phi_0 + \Delta \Phi(\Theta)}(y_t \mid x, y_{< t})\right) $

Mientras que $x$ e $y$, en la primera expresión genérica representan cualquier posible secuencia de entrada y salida, en la segunda expresión representan únicamente las secuencias de entrada y salida del conjunto de datos usados para el reentrenamiento.

### 2.2. Full-Rank Matrix

Una matriz puede verse como un conjunto de columnas, y cada columna como un vector, de forma que el conjunto de columnas genera un subespacio llamado _espacio columna_.

Al multiplicar una matriz $\mathbb{R}^{d \times k}$ por un vector $\mathbb{R}^{k}$ , se transforma el vector del espacio de entrada $\mathbb{R}^{k}$ al espacio de salida $\mathbb{R}^{d}$. El nuevo vector pertenece al _espacio columna_ de la matriz, que es un subespacio de $\mathbb{R}^{d}$. La dimensión de este espacio columna se conoce como "_rango_" de la matriz.

Se aspira a que el rango de la matriz sea lo más grande posible, para que aproveche el máximo número de dimensiones. Porque si dos vectores columna son iguales, o alguno es una combinación lineal de otro, entonces se reducen los grados de libertad. Se reduce el rango de la matriz. Es como estar en un espacio _3D_, pero limitar todas las acciones a un plano _2D_ dentro de dicho espacio _3D_.

Las matrices de pesos preentrenadas de los grandes modelos de lenguaje suelen ser de "_rango completo_", pero el cambio efectivo en los pesos $\Delta$ necesario para adaptar el modelo a una nueva tarea específica a menudo reside en un subespacio de mucha menor dimensión. Es decir, $\Delta$ puede ser aproximado por una matriz de "_bajo rango_".

### 2.3. Low-Rank Decomposition

Durante la inferencia, la salida de una capa lineal es el producto de su matriz de pesos preentrenados $W_0$ por la entrada $x$.

- $ h = W_0 x $

- $ W_0 \in \mathbb{R}^{d \times k} $

_LoRA_ propone añadir nuevo conocimiento al modelo añadiendo una nueva matriz de pesos $\Delta W$ de rango bajo entrenada sobre un nuevo conjunto de datos.

- $ h = W_0 x + \Delta W x $

Construyendo la nueva matriz como una composición de dos matrices $A$ y $B$.

- $ \Delta W = B A $

- $ B \in \mathbb{R}^{d \times r} $

- $ A \in \mathbb{R}^{r \times k} $

La matriz $A$ transforma la entrada a un espacio intermedio de baja dimensión $r$. Y la matriz $B$ transforma el vector al espacio de salida de dimensión $d$. La dimensión interna $r$ (el rango de la descomposición) puede ser tan pequeña como se quiera, siempre que se cumpla $r \ll \min(d, k)$.

El _paper_ menciona que incluso valores de $r$ bajos como $1$ ó $2$ pueden llegar a dar buenos resultados.

### 2.4. Entrenamiento

Durante el entrenamiento, la matriz $W_0$ se congela, y los únicos parámetros que se entrenan son las matrices $A$ y $B$.

La matriz $A$ se inicializa con valores aleatorios de una distribución normal, y $B$ con ceros.

Y los valores de $\Delta W x$ se escalan por $\dfrac{\alpha}{r}$, siendo $\alpha$ una constante.

El _paper_ recomienda cambiar sólo las capas relevantes del modelo, en particular las matrices $W_Q$ y $W_V$ del mecanismo de atención.

Una vez acabado el entrenamiento, las matrices entrenadas se pueden añadir al proceso de inferencia para transferir el nuevo conocimiento adquirido al modelo. De hecho, el incremento $\Delta W = B A$ se puede sumar directamente a la matriz $W_0$ para no introducir ningún tipo de latencia al modelo original.

Algunas líneas relevantes del código fuente:

```python
self.lora_A = nn.Parameter(self.weight.new_zeros((r, in_features)))
self.lora_B = nn.Parameter(self.weight.new_zeros((out_features, r)))
self.scaling = self.lora_alpha / self.r

nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
nn.init.zeros_(self.lora_B)

self.weight.data += T(self.lora_B @ self.lora_A) * self.scaling
```